In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from sys import argv,exit
import os
from pathlib import Path
import re


In [4]:
sched_data

TRAINID  TRAINNUMBER  SEQNUMBER STTNCODE   BLCKSCTN  WTTARVL  WTTDPRT  \
0         2         5676          1      Dmr   Dmr-Pasg    20400  20400.0   
1         2         5676          2     Pasg   Pasg-Pec    21120  21180.0   
2         2         5676          3      Pec   Pec-Kugt    22140  22200.0   
3         2         5676          4     Kugt  Kugt-Nlkt    23400  23520.0   
4         2         5676          5     Nlkt  Nlkt-Manu    24540  24600.0   
..      ...          ...        ...      ...        ...      ...      ...   
94        7        12098         11     Tlmr  Tlmr-Jrna    78060  78060.0   
95        7        12098         12     Jrna  Jrna-Jgnr    79620  79620.0   
96        7        12098         13     Jgnr  Jgnr-Agtl    80640  80640.0   
97        7        12098         14     Agtl       Agtl    81000      NaN   
98        8         7030          1      Dmr   Dmr-Pasg    85500  85620.0   

    WTTNEXTARVL  WTTDAYOFRUN  RUNTIME  ...  CREWCHNGCODE  LOCOCHANGE  \
0       20520.0            1    120.0  ...           NaN         NaN   
1       22140.0            1   1560.0  ...           NaN         NaN   
2       23400.0            1   1200.0  ...           NaN         NaN   
3       24540.0            1   1020.0  ...           NaN         NaN   
4       25500.0            1    900.0  ...           NaN         NaN   
..          ...          ...      ...  ...           ...         ...   
94      79620.0            1      NaN  ...           NaN         NaN   
95      80640.0            1      NaN  ...           NaN         NaN   
96      81000.0            1      NaN  ...           NaN         NaN   
97          NaN            1      NaN  ...           NaN         NaN   
98      86400.0            1      NaN  ...           NaN         NaN   

    TRTNCODE  GARBG  WATER  STTNLINE  PFDRTN  CLASSFLAG  REPORTINGFLAG  \
0        NaN    NaN    NaN       NaN     NaN        NaN            NaN   
1        NaN    NaN    NaN       NaN     NaN        NaN            NaN   
2        NaN    NaN    NaN       NaN     NaN        NaN            NaN   
3        NaN    NaN    NaN       NaN     NaN        NaN            NaN   
4        NaN    NaN    NaN       NaN     NaN        NaN            NaN   
..       ...    ...    ...       ...     ...        ...            ...   
94       NaN    NaN    NaN       NaN     NaN        NaN            NaN   
95       NaN    NaN    NaN       NaN     NaN        NaN            NaN   
96       NaN    NaN    NaN       NaN     NaN        NaN            NaN   
97       NaN    NaN    NaN       NaN     NaN        NaN            NaN   
98       NaN    NaN    NaN       NaN     NaN        NaN            NaN   

    BLCKSCTNLINE  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN  
..           ...  
94           NaN  
95           NaN  
96           NaN  
97           NaN  
98           NaN  

[99 rows x 49 columns]

In [6]:
sched_data["STTNCODE"].unique()

array(['Dmr', 'Pasg', 'Pec', 'Kugt', 'Nlkt', 'Manu', 'Skap', 'Jwnr',
       'Absa', 'Mgkm', 'Tlmr', 'Jrna', 'Jgnr', 'Agtl'], dtype=object)

In [64]:
def generate_dt_data(Arr,Dep,Dist):
    """ Given --> (Arrival | Depart | Dist) Series/List
      Returns time and distance for plotting D-T graph"""
    assert len(Arr) == len(Dep) and len(Arr) == len(Dist)
    time = []
    dist = []

    for i in range(len(Arr)):
        time.append(Arr[i])
        dist.append(Dist[i])
        time.append(Dep[i])
        dist.append(Dist[i])

    return time,dist

def to_MM(HHMMSS):
    hh = 0
    mm = 0
    ss = 0
    if HHMMSS != 0:
        HHMMSS = str(HHMMSS)
        ss = int(HHMMSS[-2:])
        if len(HHMMSS) > 2:
            mm = int(HHMMSS[-4:-2])
        if len(HHMMSS) > 4:
            hh = int(HHMMSS[:-4])

    return hh*60+mm+ss/60


def day_pattern(day):
    pat = list('.,.,.,.,.,.,.')
    pat[2*(day-1)] = '1'
    pat = "".join(pat)
    return pat

def HHMM(x):                # x is in minutes
    HH = str(x // 60)
    MM = str(x % 60)
    if len(HH) == 1:
        HH = '0'+HH
    if len(MM) == 1:
        MM = '0'+MM
    return str(HH)+str(MM)

loop_occs = {}
def decide_loop(stsn,nextblock,arr,dep):
    if int(str(nextblock)[-4]) == 0:                 # train going up
        if arr == dep:
            if loop_refs[stsn].upML not in loop_occs.keys():
                loop_occs.update({loop_refs[stsn].upML:[(arr,dep)]})
            else:
                loop_occs[loop_refs[stsn].upML].append((arr,dep))
            return loop_refs[stsn].upML
        else:
            loop_alloted = False
            if len(loop_refs[stsn].uploop) != 0:
                for loop in loop_refs[stsn].uploop:
                    if loop not in loop_occs.keys():
                        loop_alloted = True
                        loop_occs.update({loop:[(arr,dep)]})
                        return loop
                    else:
                        loop_ok = True
                        for i,occs in enumerate(loop_occs[loop]):
                            if (occs[0] <= arr and arr <= occs[1]) or (occs[0] <= dep and dep <= occs[1]):
                                loop_ok = False
                                break
                        if loop_ok == False:
                            continue
                        else:
                            loop_alloted = True
                            loop_occs[loop].append((arr,dep))
                            return loop    
            if len(loop_refs[stsn].commonloop) != 0 and loop_alloted == False:
                for loop in loop_refs[stsn].commonloop:
                    if loop not in loop_occs.keys():
                        loop_alloted = True
                        loop_occs.update({loop:[(arr,dep)]})
                        return loop
                    else:
                        loop_ok = True
                        for occs in loop_occs[loop]:
                            if (occs[0] <= arr and arr <= occs[1]) or (occs[0] <= dep and dep <= occs[1]):
                                loop_ok = False
                                break
                        if loop_ok == False:
                            continue
                        else:
                            loop_occs[loop].append((arr,dep))
                            return loop 

# goes to else only if len of common loop and uploop list is 0. change this s.t. else is run when loop not alloted yet.

            else:
                if loop_refs[stsn].upML not in loop_occs.keys():
                    loop_occs.update({loop_refs[stsn].upML:[(arr,dep)]})
                else:
                    for occs in loop_occs[loop_refs[stsn].upML]:
                        if (occs[0] <= arr and arr <= occs[1]) or (occs[0] <= dep and dep <= occs[1]):
                            break
                    loop_occs[loop_refs[stsn].upML].append((arr,dep))
                return loop_refs[stsn].upML

                
    else:
        if arr == dep:
            if loop_refs[stsn].downML not in loop_occs.keys():
                loop_occs.update({loop_refs[stsn].downML:[(arr,dep)]})
            else:
                loop_occs[loop_refs[stsn].downML].append((arr,dep))
            return loop_refs[stsn].downML
        else:
            if len(loop_refs[stsn].downloop) != 0:
                for loop in loop_refs[stsn].downloop:
                    if loop not in loop_occs.keys():
                        loop_occs.update({loop:[(arr,dep)]})
                        return loop
                    else:
                        loop_ok = True
                        for occs in loop_occs[loop]:
                            if (occs[0] <= arr and arr <= occs[1]) or (occs[0] <= dep and dep <= occs[1]):
                                loop_ok = False
                                break
                        if loop_ok == False:
                            continue
                        loop_occs[loop].append((arr,dep))
                        return loop  
            if len(loop_refs[stsn].commonloop) != 0:
                for loop in loop_refs[stsn].commonloop:
                    if loop not in loop_occs.keys():
                        loop_occs.update({loop:[(arr,dep)]})
                        return loop
                    else:
                        loop_ok = True
                        for occs in loop_occs[loop]:
                            if (occs[0] <= arr and arr <= occs[1]) or (occs[0] <= dep and dep <= occs[1]):
                                loop_ok = False
                                break
                        if loop_ok == False:
                            continue
                        loop_occs[loop].append((arr,dep))
                        return loop
            else:
                if loop_refs[stsn].downML not in loop_occs.keys():
                    loop_occs.update({loop_refs[stsn].downML:[(arr,dep)]})
                else:
                    for occs in loop_occs[loop_refs[stsn].downML]:
                        if (occs[0] <= arr and arr <= occs[1]) or (occs[0] <= dep and dep <= occs[1]):
                            break
                    loop_occs[loop_refs[stsn].downML].append((arr,dep))
                return loop_refs[stsn].downML

class station:

    def __init__(self,Looplist) -> None:

        self.uploop = list()
        self.downloop = list()
        self.commonloop = list()
        self.upML = None
        self.downML = None

        for loop in Looplist:
            if int(str(loop)[-4]) == 0:                                           # dir -> up
                if int(str(loop)[-3]) == 0:                                       # ML
                    self.upML = loop
                else:                                                             # loop
                    self.uploop.append(loop)
            elif int(str(loop)[-4]) == 1:                                         # dir -> down
                if int(str(loop)[-3]) == 0:                                       # ML
                    self.downML = loop
                else:                                                             # loop
                    self.downloop.append(loop)
            else:                                                                 # dir -> common
                self.commonloop.append(loop)

        self.uploop.sort()
        self.downloop.sort()
        self.commonloop.sort()
        

pyth_dir = Path.cwd()
sim_in = pyth_dir.parent / 'simulator_input'
data_dir = pyth_dir.parent / 'Railway data for ROU BSP'
stations = pd.read_csv('./station.txt', delim_whitespace=True)
loops = pd.read_csv('./loop.txt',delim_whitespace=True, index_col=False)
blocks = pd.read_csv('./block.txt',delim_whitespace=True, index_col=False)
# print("LOOOPs---------------------")
# print(loops.head(10).T)
stsn = dict()
stations = pd.read_csv('./station.txt', delim_whitespace=True)
for indx in range(stations.shape[0]):
    stsn.update({stations.loc[indx,'/*Name'] : (stations.loc[indx,'startKM'] + stations.loc[indx,'endKM'])/2})

loop_refs = {}
for stncode in loops['Station-Code'].unique():
    loop_refs.update({stncode: station(loops[loops['Station-Code']==stncode]['/*Loop-Number'].tolist())})



C:\Users\nisar\AppData\Local\Temp\ipykernel_5896\1455787221.py:187: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  loops = pd.read_csv('./loop.txt',delim_whitespace=True, index_col=False)
C:\Users\nisar\AppData\Local\Temp\ipykernel_5896\1455787221.py:188: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  blocks = pd.read_csv('./block.txt',delim_whitespace=True, index_col=False)


In [65]:
block_refs = {}
for i in range(blocks.shape[0]):
    prev = stations.loc[stations['endKM']==blocks.loc[i,'Starting-Mile-Post(Km)'],'/*Name'].tolist()[0]
    next = stations.loc[stations['startKM']==blocks.loc[i,'Ending-Mile-Post(Km)'],'/*Name'].tolist()[0]
    if blocks.loc[i,'Direction'] == 'up':
        block_refs.update({prev+str('-')+next : blocks.loc[i,'/*Block-Number']})
    else:
        block_refs.update({next+str('-')+prev : blocks.loc[i,'/*Block-Number']})


In [66]:
block_refs

{'Dmr-Pasg': 10010,
 'Pasg-Pec': 20010,
 'Pec-Kugt': 30010,
 'Kugt-Nlkt': 40010,
 'Nlkt-Manu': 50010,
 'Manu-Skap': 60010,
 'Skap-Jwnr': 70010,
 'Jwnr-Absa': 80010,
 'Absa-Mgkm': 90010,
 'Mgkm-Tlmr': 100010,
 'Tlmr-Jrna': 110010,
 'Jrna-Jgnr': 120010,
 'Jgnr-Agtl': 130010}

In [67]:
def Gen_Occupancy(occupancy,verbose = False):
    """
    Takes occupancy DataFrame as input and creates Occupancy.txt in the current folder
    columns of Occupancy DataFrame should be:
    'STTNCODE','BLCKSCTN','WTTARVL','WTTDPRT','WTTNEXTARVL'
    """

    occ = open('Maintenance.txt','w')
    occ.write('/*Loop/Block-Number Start-time End-time Allowed-Direction*/ \n\n')
   
    for i in range(occupancy.shape[0]):
        try:
            loopline = decide_loop(occupancy.loc[i,'STTNCODE'],block_refs[occupancy.loc[i,'BLCKSCTN']],occupancy.loc[i,'WTTARVL'],occupancy.loc[i,'WTTDPRT'])
            occ.write(str(loopline)+' "'+str(occupancy.loc[i,'WTTARVL'])+'" "'+str(occupancy.loc[i,'WTTDPRT'])+'" '+'"none" \n')
            if not pd.isna(occupancy.loc[i,'BLCKSCTN']):
                occ.write(str(block_refs[occupancy.loc[i,'BLCKSCTN']])+' "'+str(occupancy.loc[i,'WTTDPRT'])+'" "'+str(occupancy.loc[i,'WTTNEXTARVL'])+'" '+'"none" \n')
        except KeyError:
                print("Error aa gya")
                if verbose == True:
                    print(occupancy.loc[i,'BLCKSCTN'], 'Not in Infrastructure')


    occ.close()

# days = [7,1,2]
# for i,d in enumerate(days):
#     pattern = day_pattern(d)
#     mask = list()
#     for indx in range(sched_data.shape[0]):
#         mask.append(bool(re.search(pattern,sched_data.loc[indx,'BLCKBUSYDAYS'])))

#     day = sched_data[mask].copy()
#     day['TRAINID'] = str(d) + day['TRAINID'].astype(str)
#     day[['WTTARVL','WTTDPRT','WTTNEXTARVL']] = day[['WTTARVL','WTTDPRT','WTTNEXTARVL']].apply(lambda x : x % 86400 + i*86400) 

#     if i == 0:
#         combined = day
#     else:
#         combined = pd.concat([combined,day])



# occupancy = combined[['TRAINNUMBER','TRAINID','STTNCODE','BLCKSCTN','WTTARVL','WTTDPRT','WTTNEXTARVL']].copy()
# occupancy[['WTTARVL','WTTDPRT','WTTNEXTARVL']] = combined[['WTTARVL','WTTDPRT','WTTNEXTARVL']].apply(lambda x : x/60)
# occupancy.reset_index(drop = True,inplace = True)

def between(ele,R1,R2):
    if ele > R1  and ele < R2:
        return True
    else: 
        return False

def Create_conflic_report(occupancy):
    conflic_file = open('Train Conflicts in Scheduled data.txt','w')
    num_conflix = dict()

    for sect in occupancy['BLCKSCTN'].unique():
        conflic_file.write(f'For Section {sect} \n')
        sect_data = occupancy[occupancy['BLCKSCTN'] == sect].reset_index(drop=True)
    # display(sect_data)
        for indx in range(sect_data.shape[0]):
            trn1 = str(sect_data.loc[indx,'TRAINNUMBER'])
            trn1_strt = sect_data.loc[indx,'WTTDPRT']
            trn1_end = sect_data.loc[indx,'WTTNEXTARVL']

            for indx2 in range(indx+1,sect_data.shape[0]):
                trn2 = str(sect_data.loc[indx2,'TRAINNUMBER'])
                trn2_strt = sect_data.loc[indx2,'WTTDPRT']
                trn2_end = sect_data.loc[indx2,'WTTNEXTARVL']

                if between(trn2_strt,trn1_strt,trn1_end) or between(trn2_end,trn1_strt,trn1_end):
                    for trn in [trn1,trn2]:
                        if trn not in num_conflix.keys():
                            num_conflix.update({trn:1})
                        else:
                            num_conflix[trn] += 1
                    str1 = f'Found {trn1} time ({trn1_strt},{trn1_end}) and {trn2} time ({trn2_strt},{trn2_end}) using block {sect} simultaneously \n'
                    conflic_file.write(str1)
    conflic_file.write(f'Number of conflicting Trains : {len(num_conflix.keys())} \n')
    conflic_file.write('Trains and their number of conflicts \n')
    for trns in num_conflix.keys():
        conflic_file.write(f'{trns} : {num_conflix[trns]} \n')

    conflic_file.close()

In [68]:
def dayendtime(time):
    end = 1440
    while abs(time-end) > 1440 or time > end:
        end = end + 1440 

    return end

# def calday(daystr):
#     for day,val in enumerate(list(daystr)):
#         if val == '1':
#             return day+1

# user_input = dict()

# with open('../CRIS_train_inputs_for_IITB.csv') as f:
#     input = f.read().split('\n')
#     for val in input[:-2]:
#         vals = val.split(',')
#         user_input.update({vals[0]:vals[1]})    

# user_input.update({'Station:Halt' : input[-2].split('Halt,')[1]})
# user_input.update({'Start_times' : [int(x) for x in input[-1].split('times,')[1].removeprefix("'").removesuffix("'").split(',')]})

# trnday = calday(user_input['Dayofweek'])


sched_data['TRAINNUMBER'] = sched_data['TRAINNUMBER'].apply(lambda x : '00' + str(x) if len(str(x)) == 3 else ('0'+str(x) if len(str(x)) == 4 else str(x)))
sched_data['TRAINID'] = sched_data['WTTDAYOFRUN'].astype(str) + sched_data['TRAINID'].astype(str)
# print("Sche data")
# print(sched_data.T)
# print("-----------------------------------------------------")
days = [3,4]
for i,d in enumerate(days):
    pattern = day_pattern(d)
    mask = list()
    for indx in range(sched_data.shape[0]):
        mask.append(bool(re.search(pattern,sched_data.loc[indx,'BLCKBUSYDAYS'])))

    day = sched_data[mask].copy()
    day['TRAINID'] = str(d) + day['TRAINID'].astype(str)
    day[['WTTARVL','WTTDPRT','WTTNEXTARVL']] = day[['WTTARVL','WTTDPRT','WTTNEXTARVL']].apply(lambda x : x % 86400 + i*86400) 

    if i == 0:
        combined = day
    else:
        combined = pd.concat([combined,day])
# print("combined")
# print("-----------------------------------------------------")
# print(combined)
occupancy = combined[['TRAINNUMBER','TRAINID','STTNCODE','BLCKSCTN','WTTARVL','WTTDPRT','WTTNEXTARVL']].copy()
occupancy[['WTTARVL','WTTDPRT','WTTNEXTARVL']] = combined[['WTTARVL','WTTDPRT','WTTNEXTARVL']].apply(lambda x : round(x/60,1))         # rounding
occupancy.reset_index(drop = True,inplace = True)

for indx in range(occupancy.shape[0]):
    if occupancy.loc[indx,'WTTARVL'] > occupancy.loc[indx,'WTTDPRT']:
        occupancy.loc[indx,'WTTDPRT'] = dayendtime(occupancy.loc[indx,'WTTARVL'])
    if occupancy.loc[indx,'WTTDPRT'] > occupancy.loc[indx,'WTTNEXTARVL']:
        occupancy.loc[indx,'WTTNEXTARVL'] = dayendtime(occupancy.loc[indx,'WTTDPRT'])

loop_occs = {}
Gen_Occupancy(occupancy)

Error aa gya
